# KNN for images ----> by **Ido Fuchs** & **Shiraz Toledano**

#### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import os
import warnings
warnings.filterwarnings("ignore")

#### Building the algorithm

In [ ]:
def get_args():
    parser = argparse.ArgumentParser(description='KNN classifier')
    parser.add_argument('-i', '--input', type=str, help='Input file')
    parser.add_argument('-k', '--k', type=int, help='Number of neighbors')
    parser.add_argument('-o', '--output', type=str, help='Output file')
    args = parser.parse_args()
    return args